In [1]:
# Parameters
cpu = 8
group_name = "Astro-TE_NN"
mem_gb = 10


In [2]:
import pandas as pd
from scipy.stats import norm, zscore
import numpy as np
import seaborn as sns
import subprocess
import matplotlib.pyplot as plt 

In [3]:
#group_name = 'DG_Glut'

In [4]:
home_dir = '/home/qzeng_salk_edu/project/241018_pre_ml'
ct = group_name

In [5]:
loop_dir = '/data/female-amb/Diff.Result/loop_ct_age/diff'

In [6]:
loopq = pd.read_hdf(f'{loop_dir}/{ct}/loop_Q.hdf', key='data')
loopt = pd.read_hdf(f'{loop_dir}/{ct}/loop_T.hdf', key='data')

loopq.columns = [f"{ct}.2mo.Q", f"{ct}.9mo.Q", f"{ct}.18mo.Q"]
loopt.columns = [f"{ct}.2mo.T", f"{ct}.9mo.T", f"{ct}.18mo.T"]

In [7]:
loopall = pd.read_hdf(f"{loop_dir}/{ct}/merged_loop.hdf")
loopall = pd.concat([loopall, loopq, loopt], axis = 1)
loopall.head()

,0,1,2,3,4,5,Qanova,Eanova,Tanova,Astro-TE_NN.2mo.Q,Astro-TE_NN.9mo.Q,Astro-TE_NN.18mo.Q,Astro-TE_NN.2mo.T,Astro-TE_NN.9mo.T,Astro-TE_NN.18mo.T
0,chr1,4480000,4490000,chr1,4750000,4760000,0.183779,1.465435,1.146258,0.004058,0.003762,0.003847,0.274210,0.218420,0.228002
1,chr1,4480000,4490000,chr1,4760000,4770000,0.799520,0.909355,0.632955,0.004602,0.003975,0.004071,0.294052,0.248998,0.269647
2,chr1,4490000,4500000,chr1,4750000,4760000,0.082422,1.363373,0.463700,0.004666,0.004642,0.004455,0.300205,0.262034,0.274437
3,chr1,4490000,4500000,chr1,4760000,4770000,0.054755,0.984268,0.228385,0.004900,0.005079,0.004913,0.315871,0.295557,0.322144
4,chr1,4500000,4510000,chr1,4740000,4750000,0.082489,1.435242,0.298939,0.004068,0.003965,0.004174,0.226451,0.208471,0.239040


In [8]:
sell = ((zscore(np.log(loopall['Qanova']))>norm.isf(0.2)) & 
        (zscore(np.log(loopall['Tanova']))>norm.isf(0.2)) )
sell.sum()

29515

In [9]:
loopall['Diff_Loop'] = 0
loopall.loc[sell, 'Diff_Loop'] = 1
loopall.to_csv(f"{ct}.anchor1_loop.bed", sep = '\t', header = False, index = False)

In [10]:
# move columns 3 4 5 in to the first three
cols = loopall.columns.tolist()
cols = cols[3:6] + cols[:3] + cols[6:] 
loopall = loopall[cols]
loopall.to_csv(f"{ct}.anchor2_loop.bed", sep = '\t', header = False, index = False)

In [11]:
# so diff loop as two anchors, so two chro:start-end region, i want to get the intersection with another bed file as long as it intersect with eithe ranhocr
command = f"bedtools intersect -a {ct}.anchor1_loop.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.anchor1_loop_gene.bed"
subprocess.run(command, shell=True)

command = f"bedtools intersect -a {ct}.anchor2_loop.bed -b {home_dir}/mm10_gene_2kb.bed -wa -wb > {ct}.anchor2_loop_gene.bed"
subprocess.run(command, shell=True)

CompletedProcess(args='bedtools intersect -a Astro-TE_NN.anchor2_loop.bed -b /home/qzeng_salk_edu/project/241018_pre_ml/mm10_gene_2kb.bed -wa -wb > Astro-TE_NN.anchor2_loop_gene.bed', returncode=0)

In [12]:
subprocess.run(f"rm {ct}.anchor1_loop.bed", shell=True)
subprocess.run(f"rm {ct}.anchor2_loop.bed", shell=True)

CompletedProcess(args='rm Astro-TE_NN.anchor2_loop.bed', returncode=0)

In [13]:
# load the output files and concat
anchor1 = pd.read_csv(f"{ct}.anchor1_loop_gene.bed", sep = '\t', header = None)
anchor2 = pd.read_csv(f"{ct}.anchor2_loop_gene.bed", sep = '\t', header = None)
# concat
anchor = pd.concat([anchor1, anchor2], axis = 0)

In [14]:
anchor.columns = ['chrom','anchor1_start','anchor1_end','chrom','anchor2_start','anchor2_end',
                'Qanova','Eanova','Tanova','2mo.Q','9mo.Q','18mo.Q','2mo.T','9mo.T','18mo.T','Diff_Loop',
                'gene_chrom','gene_start','gene_end','gene_id','strand','gene_name','gene_type']
anchor

,chrom,anchor1_start,anchor1_end,chrom,anchor2_start,anchor2_end,Qanova,Eanova,Tanova,2mo.Q,...,9mo.T,18mo.T,Diff_Loop,gene_chrom,gene_start,gene_end,gene_id,strand,gene_name,gene_type
0,chr1,4480000,4490000,chr1,4750000,4760000,0.183779,1.465435,1.146259,0.004058,...,0.218420,0.228002,0,chr1,4488931,4499354,ENSMUSG00000025902.9,-,Sox17,protein_coding
1,chr1,4480000,4490000,chr1,4760000,4770000,0.799520,0.909355,0.632955,0.004602,...,0.248998,0.269647,0,chr1,4488931,4499354,ENSMUSG00000025902.9,-,Sox17,protein_coding
2,chr1,4490000,4500000,chr1,4750000,4760000,0.082422,1.363373,0.463700,0.004666,...,0.262034,0.274437,0,chr1,4494551,4501558,ENSMUSG00000104238.1,+,RP23-285G23.2,processed_transcript
3,chr1,4490000,4500000,chr1,4750000,4760000,0.082422,1.363373,0.463700,0.004666,...,0.262034,0.274437,0,chr1,4488931,4499354,ENSMUSG00000025902.9,-,Sox17,protein_coding
4,chr1,4490000,4500000,chr1,4760000,4770000,0.054755,0.984268,0.228385,0.004900,...,0.295557,0.322144,0,chr1,4494551,4501558,ENSMUSG00000104238.1,+,RP23-285G23.2,processed_transcript
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340748,chr19,60520000,60530000,chr19,60420000,60430000,2.817003,5.021049,1.330845,0.009071,...,0.241299,0.306562,0,chr19,60522696,60583023,ENSMUSG00000033417.11,-,Cacul1,protein_coding
340749,chr19,60520000,60530000,chr19,60430000,60440000,1.125436,3.943785,0.830614,0.011542,...,0.222674,0.276658,0,chr19,60522696,60583023,ENSMUSG00000033417.11,-,Cacul1,protein_coding
340750,chr19,60520000,60530000,chr19,60440000,60450000,1.045204,4.186615,1.426995,0.012078,...,0.234325,0.303569,0,chr19,60522696,60583023,ENSMUSG00000033417.11,-,Cacul1,protein_coding
340751,chr19,60520000,60530000,chr19,60450000,60460000,0.573376,2.861494,2.225769,0.011378,...,0.164777,0.233473,0,chr19,60522696,60583023,ENSMUSG00000033417.11,-,Cacul1,protein_coding


In [15]:
subprocess.run(f"rm {ct}.anchor1_loop_gene.bed", shell=True)
subprocess.run(f"rm {ct}.anchor2_loop_gene.bed", shell=True)

CompletedProcess(args='rm Astro-TE_NN.anchor2_loop_gene.bed', returncode=0)

In [16]:
# add a distance columns, anchor1_start to gene_start and gene end, use which ever is closer
anchor['anchor1_distance'] = np.minimum(abs(anchor['anchor1_start'] - anchor['gene_start']), abs(anchor['anchor1_start'] - anchor['gene_end']))
anchor['anchor2_distance'] = np.minimum(abs(anchor['anchor2_start'] - anchor['gene_start']), abs(anchor['anchor2_start'] - anchor['gene_end']))
anchor['gene_length'] = anchor['gene_end'] - anchor['gene_start']
# drop chrom	anchor1_start	anchor1_end	chrom	anchor2_start	anchor2_end	, 'gene_chrom', 'gene_start','gene_end', 'strand', 'gene_type'
anchor = anchor.drop(columns = ['chrom','anchor1_start','anchor1_end','chrom','anchor2_start','anchor2_end','gene_chrom', 'gene_start','gene_end', 'strand', 'gene_type'])
anchor

,Qanova,Eanova,Tanova,2mo.Q,9mo.Q,18mo.Q,2mo.T,9mo.T,18mo.T,Diff_Loop,gene_id,gene_name,anchor1_distance,anchor2_distance,gene_length
0,0.183779,1.465435,1.146259,0.004058,0.003762,0.003847,0.274210,0.218420,0.228002,0,ENSMUSG00000025902.9,Sox17,8931,250646,10423
1,0.799520,0.909355,0.632955,0.004602,0.003975,0.004071,0.294052,0.248998,0.269647,0,ENSMUSG00000025902.9,Sox17,8931,260646,10423
2,0.082422,1.363373,0.463700,0.004666,0.004642,0.004455,0.300205,0.262034,0.274437,0,ENSMUSG00000104238.1,RP23-285G23.2,4551,248442,7007
3,0.082422,1.363373,0.463700,0.004666,0.004642,0.004455,0.300205,0.262034,0.274437,0,ENSMUSG00000025902.9,Sox17,1069,250646,10423
4,0.054755,0.984268,0.228385,0.004900,0.005079,0.004913,0.315871,0.295557,0.322144,0,ENSMUSG00000104238.1,RP23-285G23.2,4551,258442,7007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
340748,2.817003,5.021049,1.330845,0.009071,0.008548,0.010494,0.276966,0.241299,0.306562,0,ENSMUSG00000033417.11,Cacul1,2696,102696,60327
340749,1.125436,3.943785,0.830614,0.011542,0.010661,0.012195,0.253496,0.222674,0.276658,0,ENSMUSG00000033417.11,Cacul1,2696,92696,60327
340750,1.045204,4.186615,1.426995,0.012078,0.011027,0.012436,0.277992,0.234325,0.303569,0,ENSMUSG00000033417.11,Cacul1,2696,82696,60327
340751,0.573376,2.861494,2.225769,0.011378,0.011132,0.012139,0.243311,0.164777,0.233473,0,ENSMUSG00000033417.11,Cacul1,2696,72696,60327


In [17]:
anchor.to_csv(f"{home_dir}/ml_input/{ct}/{ct}.Loop_gene.csv",  header = True, index = False)

In [18]:
import subprocess
#ct = 'L6_IT_CTX_Glut'
#subprocess.run(f"tar -cjvf ml_input/{ct}/{ct}.Loop_gene.csv.gz ml_input/{ct}/{ct}.Loop_gene.csv", shell=True)
subprocess.run(f"gzip -k {home_dir}/ml_input/{ct}/{ct}.Loop_gene.csv", shell=True)

CompletedProcess(args='gzip -k /home/qzeng_salk_edu/project/241018_pre_ml/ml_input/Astro-TE_NN/Astro-TE_NN.Loop_gene.csv', returncode=0)

In [19]:
subprocess.run(f"rm {home_dir}/ml_input/{ct}/{ct}.Loop_gene.csv", shell=True)

CompletedProcess(args='rm /home/qzeng_salk_edu/project/241018_pre_ml/ml_input/Astro-TE_NN/Astro-TE_NN.Loop_gene.csv', returncode=0)

In [20]:
#anchor.to_csv(f"ml_input/{ct}.diff_loop_gene.csv",  header = True, index = False)